In [1]:
from instance_reader import read_instance
from Low_levels_copy import *
import numpy as np
import random
import time
from funciones_auxiliares import seleccionar_segun_probabilidad

# instancia

In [20]:
instance = read_instance("datasets/a/instance_0007.txt")

# Low Levels

In [21]:
low_level1 = LowLevel3_factibilizadora(id=0, nombre= "LowLevel1")

# Definimos una lista de niveles bajos

low_levels = [low_level1]#, low_level12]

In [22]:
sol_inicial = instance.constructora2()

In [23]:
len(sol_inicial.infesible_type()[2])

8

In [28]:
len(sol_inicial.selected_runners)

180

In [25]:
sol_nueva = low_level1.implementacion(sol_inicial)
len(sol_nueva.infesible_type()[2])

1171


8

In [26]:
len(sol_nueva.id_selected_runners)

180

In [8]:
class HiperHeuristica():
    '''Objeto que aplica el algoritmo'''
    def __init__(self, instancia = Instance, V = float, low_levels = list):
        self.instancia = instancia # objeto instancia
        self.V = V #
        self.low_levels = low_levels # lista con low levels
        self.n = len(self.low_levels) # int que tiene el número de low levels
        self.P = np.ones((self.n,self.n)) # Matriz de contador mejora
        self.T = np.zeros((self.n,self.n)) # Matriz transicion mejora
        self.Q = np.ones((self.n, 2)) # Matriz de contador freno
        self.S = np.zeros((self.n, 2)) # Matriz de probabilidad de freno
        self.mejor_sol = self.instancia.constructora2()
        self.candidata = self.mejor_sol
        self.solucion_temporal = self.mejor_sol
        self.secuencia = []
        self.s = instancia.ub
        #self.soluciones_factibles = []
        self.tiempo_maximo = time.time() + 600 # minutos maximos
        self.costo_inicial = self.candidata.objective_value - self.candidata.costo_infactible()
        self.costo_mejor_solucion = self.mejor_sol.objective_value - self.mejor_sol.costo_infactible()

    def actualizar_matrices_T_S(self):
        ''' Descripción: 
        Método que actualiza las matrices 
            Args : 
            *   None
            Return : 
            *   None
            '''
        # primero inicializamos la matriz 
        for i in range(self.n):
                suma_P = sum(self.P[i])
                suma_Q = sum(self.Q[i])
                self.T[i] = self.P[i] / suma_P
                self.S[i] = self.Q[i] / suma_Q

    def implementar(self):
        self.actualizar_matrices_T_S()
        i_last = random.choice(self.low_levels)
        id_last =  i_last.id
        secuencia = []
        secuencia.append(id_last)
        
        tiempo_inicio = time.time()
        
        it = 0
        
        while self.tiempo_maximo - time.time() > 0:
            it+=1
            id_next = seleccionar_segun_probabilidad(self.T[id_last])
            secuencia.append(id_next)
            # ahora elegimos el valor u
            u_next = seleccionar_segun_probabilidad(self.S[id_last])

            if u_next == 1:
                sol_temporal = self.candidata
                for id in secuencia:
                    low_level = self.low_levels[id]
                    sol_temporal = low_level.implementacion(sol_temporal)

                if  self.condicion_1(sol_temporal, self.candidata) or self.condicion_2(sol_temporal, self.mejor_sol, tiempo_inicio, self.V):
                    self.candidata = copy.deepcopy(sol_temporal)
                    for id in range(len(secuencia)-1):
                        self.P[secuencia[id], secuencia[id+1]] = self.P[secuencia[id], secuencia[id+1]] + 1
                        self.Q[secuencia[id], 0] = self.Q[secuencia[id], 0] + 1
                    self.Q[secuencia[-1], 1] = self.Q[secuencia[-1], 1] + 1
                    
                    self.actualizar_matrices_T_S()
                
                if self.condicion_1(sol_temporal, self.mejor_sol):
                    self.mejor_sol = copy.deepcopy(sol_temporal)
                
                print(f"Conteo = {it}, secuencia = {secuencia}, s_t = {sol_temporal.objective_value:.2f} {sol_temporal.is_factible}, s_c = {self.candidata.objective_value:.2f} {self.candidata.is_factible}, s* = {self.mejor_sol.objective_value:.2f} {self.mejor_sol.is_factible}") 
                secuencia = []
                
            id_last = id_next
                    
                

    def condicion_1(self, sol_temporal: Solucion, sol_candidata : Solucion):
        if sol_temporal.objective_value - self.s*sol_temporal.costo_infactible() > sol_candidata.objective_value - self.s*sol_candidata.costo_infactible():
            return True
        return False
    
    def condicion_2(self, sol_temporal: Solucion, mejor_sol: Solucion, tiempo_inicio: float, V: float):
        time_el = time.time()-tiempo_inicio #Elapsed time
        
        if mejor_sol.is_factible: #Calculo de threshold según factibilidad de mejor solución
            rho = 10**-5 + V*(1-(time_el)/self.tiempo_maximo)
        else:
            rho = 10**-3
        
        if sol_temporal.objective_value - self.s*sol_temporal.costo_infactible()  > (1 + rho)*(mejor_sol.objective_value - self.s*mejor_sol.costo_infactible()):
            return True
        return False
        
        
        

In [9]:
hiper = HiperHeuristica(instancia= instance, V= 0.05, low_levels= low_levels)

hiper.implementar()

Conteo = 1, secuencia = [0, 0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 4, secuencia = [0, 0, 0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 6, secuencia = [0, 0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 7, secuencia = [0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 8, secuencia = [0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 13, secuencia = [0, 0, 0, 0, 0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 15, secuencia = [0, 0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 17, secuencia = [0, 0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 19, secuencia = [0, 0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 21, secuencia = [0, 0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 22, secuencia = [0], s_t = 12.35 False, s_c = 12.35 False, s* = 12.35 False
Conteo = 26, secuencia = [0, 0, 0,

KeyboardInterrupt: 

In [ ]:
hiper.P

In [ ]:
epocs = 100000 # número de veces que ?
solucion_inicial_2 = instance.constructora2()

V = 0.05

n = len(low_levels)

h = 10

P = np.ones((n, n))
T = np.zeros((n, n))

Q = np.ones((n, 2))
S = np.zeros((n, 2))

# inicializamos la matriz T_ij con P_ij / sum_{k in low_levels} P_ik
for i in range(n):
    suma_P = sum(P[i])
    suma_Q = sum(Q[i])
    T[i] = P[i] / suma_P
    S[i] = Q[i] / suma_Q

solucion_inicial = solucion_inicial_2 #Se comienza con una solución inicial s^c
mejor_solucion = solucion_inicial_2 #Se comienza con una mejor solución s*

secuencia = [] #Se inicia una secuencia vacía
matriz_secuencias = []

i_last = random.choice(low_levels)

sum_epocs = 0

soluciones_factibles = []
costo_inicial = solucion_inicial.objective_value - solucion_inicial.costo_infactible()
costo_mejor_solucion = mejor_solucion.objective_value - mejor_solucion.costo_infactible()

print(mejor_solucion.objective_value)
for count in range(epocs):
    id_i_next = seleccionar_segun_probabilidad(T[i_last.id-1]) #Setea la siguiente low_level
    i_next = low_levels[id_i_next]
    secuencia.append(id_i_next) #Agrega la siguiente low level a la lista
    u_next = random.choices([0,1], weights=S[i_next.id-1], k=1)[0] #Entrega u = 0 o 1 con probabilidad S_{i_next, u}
    
    if u_next == 1:
        solucion_temporal = i_next.implementacion(solucion_inicial) #Implementamos la low-level siguiente
        costo_temporal = solucion_temporal.objective_value - solucion_temporal.costo_infactible()
        
        if mejor_solucion.is_factible: #Calculo de threshold según factibilidad de mejor solución
            rho = 10**-5 + V*(1-len(secuencia)/epocs)
        else:
            rho = 10**-3
        
        if costo_temporal > costo_inicial or costo_temporal > (1+rho)*costo_mejor_solucion: #Condición para considerar la solución temporal como inicial
            solucion_inicial = solucion_temporal
            costo_inicial = costo_temporal
            
        if costo_temporal>costo_mejor_solucion: #Condición para considerar la solución temporal como mejor solución
            mejor_solucion = solucion_temporal
            costo_mejor_solucion = costo_temporal
            if mejor_solucion.is_factible:
                soluciones_factibles.append(mejor_solucion)
             #Actualizar la matriz P y S CREAR MÉTODO PARA ACTUALIZAR LAS MATRICES
            Q[secuencia[-1], 1] =  Q[secuencia[-1], 1] + 1
            for i in range(len(secuencia)-1):
                i_k = secuencia[i]
                i_k_1 = secuencia[i+1]
                P[i_k, i_k_1] = P[i_k, i_k_1]+1
                Q[i_k, 0] = Q[i_k, 0]+1
                for i in range(n):
                    suma_P = sum(P[i])
                    suma_Q = sum(Q[i])
                    T[i] = P[i] / suma_P
                    S[i] = Q[i] / suma_Q
        print(f"epoc: {count}, secuencia = {secuencia}, s_t = {solucion_temporal.objective_value} {solucion_temporal.is_factible}, s_c = {solucion_inicial.objective_value} {solucion_inicial.is_factible}, s* = {mejor_solucion.objective_value} {mejor_solucion.is_factible}")
        secuencia = []
    i_last = i_next
    
print(mejor_solucion)
        

In [ ]:
epocs = 100000
solucion_inicial_2 = instance.constructora2()

V = 0.5

n = len(low_levels)

h = 10

P = np.ones((n, n))
T = np.zeros((n, n))

Q = np.ones((n, 2))
S = np.zeros((n, 2))

# inicializamos la matriz T_ij con P_ij / sum_{k in low_levels} P_ik
for i in range(n):
    suma_P = sum(P[i])
    suma_Q = sum(Q[i])
    T[i] = P[i] / suma_P
    S[i] = Q[i] / suma_Q

solucion_inicial = solucion_inicial_2 #Se comienza con una solución inicial s^c
mejor_solucion = solucion_inicial_2 #Se comienza con una mejor solución s*

secuencia = [] #Se inicia una secuencia vacía
matriz_secuencias = []

i_last = random.choice(low_levels)

sum_epocs = 0

soluciones_factibles = []
costo_inicial = solucion_inicial.objective_value - solucion_inicial.costo_infactible()
costo_mejor_solucion = mejor_solucion.objective_value - mejor_solucion.costo_infactible()

print(mejor_solucion.objective_value)
for count in range(epocs):
    id_i_next = seleccionar_segun_probabilidad(T[i_last.id-1]) #Setea la siguiente low_level
    i_next = low_levels[id_i_next]
    secuencia.append(id_i_next) #Agrega la siguiente low level a la lista
    u_next = random.choices([0,1], weights=S[i_next.id-1], k=1)[0] #Entrega u = 0 o 1 con probabilidad S_{i_next, u}
    
    if u_next == 1:
        solucion_temporal = i_next.implementacion(solucion_inicial) #Implementamos la low-level siguiente
        costo_temporal = solucion_temporal.objective_value - solucion_temporal.costo_infactible()
        
        if mejor_solucion.is_factible: #Calculo de threshold según factibilidad de mejor solución
            rho = 10**-5 + V*(1-len(secuencia)/epocs)
        else:
            rho = 10**-3
            
        #Castigamos el freno si me da infactible
        if not solucion_temporal.is_factible:
            Q[secuencia[-1],1] = 0.7*Q[secuencia[-1],1]
            S[secuencia[-1]] = Q[secuencia[-1]]/sum(Q[secuencia[-1]])
        
        if costo_temporal > costo_inicial: #Condición para considerar la solución temporal como inicial
            solucion_inicial = solucion_temporal
            costo_inicial = costo_temporal
            
        if costo_temporal > (1+rho)*costo_mejor_solucion: #Condición para considerar la solución temporal como mejor solución
            mejor_solucion = solucion_temporal
            costo_mejor_solucion = costo_temporal
            if mejor_solucion.is_factible:
                soluciones_factibles.append(mejor_solucion)
             #Actualizar la matriz P y S
            Q[secuencia[-1], 1] =  Q[secuencia[-1], 1] + 1
            for i in range(len(secuencia)-1):
                i_k = secuencia[i]
                i_k_1 = secuencia[i+1]
                P[i_k, i_k_1] = P[i_k, i_k_1]+1
                Q[i_k, 0] = Q[i_k, 0]+1
                for i in range(n):
                    suma_P = sum(P[i])
                    suma_Q = sum(Q[i])
                    T[i] = P[i] / suma_P
                    S[i] = Q[i] / suma_Q
        print(f"epoc: {count}, secuencia = {secuencia}, s_t = {solucion_temporal.objective_value} {solucion_temporal.is_factible}, s_c = {solucion_inicial.objective_value} {solucion_inicial.is_factible}, s* = {mejor_solucion.objective_value} {mejor_solucion.is_factible}")
        secuencia = []
    i_last = i_next
    
print(mejor_solucion)
        

In [ ]:
mejor_solucion = None
best_value = 0
epocs = 10000

solucion_inicial_1 = instance.constructora1()
solucion_inicial_2 = instance.constructora2()
P_ij = np.ones((len(low_levels), len(low_levels)))
T_ij = np.zeros((len(low_levels), len(low_levels)))


# inicializamos la matriz T_ij con P_ij / sum_{k in low_levels} P_ik
for i in range(len(low_levels)):
    suma_P_ik = sum(P_ij[i])
    for j in range(len(low_levels)):
        T_ij[i][j] = P_ij[i][j] / suma_P_ik if suma_P_ik > 0 else 0
print(T_ij)

matriz_secuencias = []
for _ in range(epocs):
    solucion = random.choice([solucion_inicial_2])
    mejor_solucion_temporal = solucion
    best_value_temporal = solucion.objective_value
    low_level_inicial = None
    secuencia = []

    while True:
        # si no hay una low level inicial, se elige una al azar
        if low_level_inicial is None:
            low_level_elegida = random.choice(low_levels)
            id_eleccion = low_level_elegida.id
        else:
            probabilidades_siguiente_low_level = T_ij[low_level_inicial.id - 1,:]
            id_eleccion = seleccionar_segun_probabilidad(probabilidades_siguiente_low_level)
            low_level_elegida = low_levels[id_eleccion]

        secuencia.append(id_eleccion)

 
        solucion_nueva = low_level_elegida.implementacion(solucion)

        if solucion_nueva.is_factible == False: #or solucion_nueva.objective_value <= solucion.objective_value:
            break

        if low_level_inicial is None:
            pass
        else:
            if solucion_nueva.objective_value > best_value_temporal:
                mejor_solucion_temporal = solucion_nueva
                best_value_temporal = solucion_nueva.objective_value
                P_ij[low_level_inicial.id - 1][low_level_elegida.id - 1] += 1

        if low_level_inicial is None:
            id_low_level_inicial = low_level_elegida.id
           # la matriz T_ij se actualiza con el valor de P_ij/ sum_{k in low_levels} P_ik
            suma_P_ik = sum(P_ij[id_low_level_inicial - 1])
            for low_level in low_levels:
                low_level_id = low_level.id
                T_ij[id_low_level_inicial - 1][low_level_id - 1] = P_ij[id_low_level_inicial - 1][low_level_id - 1] / suma_P_ik if suma_P_ik > 0 else 0
            
        low_level_inicial = low_level_elegida
        solucion = solucion_nueva
    
    if best_value_temporal > best_value:
        mejor_solucion = solucion
        best_value = best_value_temporal
        print(f"Nueva mejor solucion encontrada:, Con mejor valor{best_value}, secuencia: {secuencia}")
    matriz_secuencias.append(secuencia)

    #print(T_ij)

print("Mejor solucion final:", mejor_solucion)

In [ ]:
mejor_solucion.is_factible

In [ ]:
1/soluciones_factibles[0].objective_value+h*soluciones_factibles[0].costo_infactible()

In [ ]:
1/soluciones_factibles[-1].objective_value+h*soluciones_factibles[-1].costo_infactible()

In [ ]:
1/mejor_solucion.objective_value + h*mejor_solucion.costo_infactible()

In [ ]:
1/mejor_solucion.objective_value + h*mejor_solucion.costo_infactible()

In [ ]:
matriz = np.zeros((3,3))
# ahora rellenamos la matriz con unos


for i in range(matriz.shape[0]):
    for j in range(matriz.shape[1]):
        matriz[i][j] = 1

print(matriz)

print("###########")

print(matriz[0,:])
print("###########")

print(sum(matriz[0]))
